### 1) Data Preparation

In [1]:
import pandas as pd
path = "C:/Users/ppirthip/OneDrive - Intel Corporation/Speckle/"
df = pd.read_csv(path+"SNR_R4_ww49.1_Speckle.csv.gz")

In [2]:
#create new column for target
# import numpy as np
# df["SPECKLE"] = np.where(df["DELTA"]==0,0,1)
# cols = df.columns.tolist()
# cols = [cols[-1]]+cols[:-1] #move speckle col to the front
# df = df.reindex(columns=cols)

In [3]:
df["DELTA"].value_counts()

0    13390
1     6317
2      129
Name: DELTA, dtype: int64

In [4]:
df["SPECKLE"].value_counts()

0    13390
1     6446
Name: SPECKLE, dtype: int64

In [ ]:
df.head()

In [ ]:
pd.options.display.max_seq_items = 2000
df.columns

In [ ]:
df.shape

In [ ]:
df.describe()

### Duplicate handling

In [5]:
def duplicates(df,keep):
    """
    Check for duplicates in VID and remove duplicates if there is any
    df[DataFrame]: input df
    keep: {‘first’, ‘last’, False}: Determines which duplicates (if any) to keep. 
    - first : Drop duplicates except for the first occurrence. 
    - last : Drop duplicates except for the last occurrence. 
    - False : Drop all duplicates.
    """
    if df['VID'].duplicated().any():
        print("There are duplicates in VID")
        df_dup = df[df.duplicated(subset=['VID'])] #get the duplicates in df 
        display(df_dup)
        df = df.drop_duplicates(subset=['VID'],keep=keep) #remove duplicates from df
        return df
    else:
        print("There are no duplicates in VID")
    

In [6]:
duplicates(df,keep="first")

There are no duplicates in VID


### 2) NA handling

In [7]:
df.shape

(19836, 1615)

In [8]:
# check column for nulls
def check_colna(df):
    """
    Check each column for nulls. Returns Feature, total null and % of null for each column
    df[DataFrame]: dataframe
    
    """
    colna_df = pd.DataFrame(columns =["Feature", "Total Null", "% of Null"])
    for col in df.columns: 
        #checking if there is any null in the column

        if df[col].isnull().sum()>0: 
            
            # if null present, total number of null in the column stores here
            total_null = df[col].isnull().sum() 
            new_row = {'Feature':col, 'Total Null':total_null, '% of Null':round(total_null*100/len(df),2)}
            #append row to the dataframe
            colna_df = colna_df.append(new_row, ignore_index=True)
            
    colna_df= colna_df.sort_values("% of Null", ascending=False)    
    return colna_df  

# check rows for nulls
def check_rowna(df,supporting_fs):  
    """
    Check each row for nulls.Returns VID, total null and % of null for each row
    df[DataFrame]: dataframe
    supporting_fs[list]: all features not used for ML except VID and SPECKLE(target)
    """
    df = df.drop(supporting_fs,axis=1)    
    colrow_df = pd.DataFrame(columns =["SPECKLE","VID", "Total Null", "% of Null"])
    for i in df.index: 
        #checking if there is any null in the row
        if df.iloc[i].isnull().sum()>0:             
            # if null present, total number of null in the row stores here
            total_null = df.iloc[i].isnull().sum() 
            new_row = {'SPECKLE':df.iloc[i,0],'VID':df.iloc[i,1], 'Total Null':total_null, '% of Null':round(total_null*100/(len(df.columns)-2),2)}
            #append row to the dataframe
            colrow_df = colrow_df.append(new_row, ignore_index=True)
            
    colrow_df= colrow_df.sort_values("% of Null", ascending=False)    
    return colrow_df   

# Drop columns based on NA threshold limit
def drop_NAcol(df,NA_limit):
    '''
    Drops columns based on proportion of NA in column
    df[DataFrame]: df
    NA_limit[float/int]: Columns with proportion of NA above NA_limit will be dropped
    '''
    threshold = len(df)*(1-NA_limit)
    df=df.dropna(axis=1, thresh=threshold)
    print(df.shape)
    return df


#### a) Check column for nulls

In [9]:
#check column na for whole df
colna_df =check_colna(df)
colna_df

,Feature,Total Null,% of Null
1230,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,16129,81.31
0,IDV_0001_SVT3GNES12_FULLDIE_SOC_0650_MED@132110,13847,69.81
399,IDV_0134_PSVT3GVTO12_FULLDIE_TALL_0950_MED@132110,13847,69.81
392,IDV_0134_PSVT3GVTO12_FULLDIE_CORE_TALL_0650_ME...,13847,69.81
393,IDV_0134_PSVT3GVTO12_FULLDIE_CORE_TALL_0950_ME...,13847,69.81
...,...,...,...
1472,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,13459,67.85
1473,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,13459,67.85
1407,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,13459,67.85
1406,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,13459,67.85


In [ ]:
# colna_df.to_csv(path+"/DataPreparation/NA_Cols.csv",index=False)

In [10]:
#check column na for speckle
colna_speckle_df =check_colna(df.loc[df['SPECKLE'] == 1])
colna_speckle_df

,Feature,Total Null,% of Null
1230,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,5268,81.73
0,IDV_0001_SVT3GNES12_FULLDIE_SOC_0650_MED@132110,4845,75.16
399,IDV_0134_PSVT3GVTO12_FULLDIE_TALL_0950_MED@132110,4845,75.16
392,IDV_0134_PSVT3GVTO12_FULLDIE_CORE_TALL_0650_ME...,4845,75.16
393,IDV_0134_PSVT3GVTO12_FULLDIE_CORE_TALL_0950_ME...,4845,75.16
...,...,...,...
1472,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,4677,72.56
1473,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,4677,72.56
1406,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,4677,72.56
1405,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,4677,72.56


In [11]:
#check column na for non-speckle
colna_nonspeckle_df =check_colna(df.loc[df['SPECKLE'] == 0])
colna_nonspeckle_df

,Feature,Total Null,% of Null
1230,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,10861,81.11
875,TPI_SIU_STATIC::SIU_STATIC_AM_K_STRESS_X_X_X_S...,9006,67.26
882,TPI_SIU_STATIC::SIU_STATIC_AM_K_STRESS_X_X_X_S...,9006,67.26
889,TPI_SIU_STATIC::SIU_STATIC_AM_K_STRESS_X_X_X_S...,9006,67.26
888,TPI_SIU_STATIC::SIU_STATIC_AM_K_STRESS_X_X_X_S...,9006,67.26
...,...,...,...
820,TPI_SIU_STATIC::SIU_STATIC_AM_K_START_X_X_X_SN...,8782,65.59
822,TPI_SIU_STATIC::SIU_STATIC_AM_K_START_X_X_X_SN...,8782,65.59
824,TPI_SIU_STATIC::SIU_STATIC_AM_K_START_X_X_X_SN...,8782,65.59
826,TPI_SIU_STATIC::SIU_STATIC_AM_K_START_X_X_X_SN...,8782,65.59


#### b) Check row for nulls

In [ ]:
supporting_fs = ['ULT@MIDAS_6261_U1', 'SORTLOT', 'SORTLOT7', 'WAFER',
       'XLOC', 'YLOC', 'ULT@MIDAS_6261_U2', 'IB@6261[CLASSHOT]',
       'FB@6261[CLASSHOT]','ARR_SPECKLE_SCREEN::PHYTON_X_USERFUNC_K_END_X_X_X_X_PBISTSPECKLE_INDICATOR::DUTCPUBAD_1@6261[CLASSHOT]',
       'TEST RESULTS', 'TEST RESULTS BITS', 'TR_BITS', 'INCOMING',
       'INCOMING BITS', 'INC_BITS', 'OUTGOING', 'OUTGOING BITS', 'OUT_BITS','DELTA']
rowna_df = check_rowna(df,supporting_fs)
rowna_df

In [ ]:
rowna_df.loc[rowna_df["% of Null"] ==100.00]

In [ ]:
# rowna_df.to_csv(path+"/DataPreparation/NA_Rows_aftercolremoval.csv",index=False)

#### c) Drop columns based on threshold limit

In [ ]:
df = drop_NAcol(df,0.8) #drop columns with >80% NA 

In [ ]:
check_colna(df)

#### d) Row NA handling

In [12]:
#check rows for NA after removing columns
supporting_fs = ['ULT@MIDAS_6261_U1', 'SORTLOT', 'SORTLOT7', 'WAFER',
       'XLOC', 'YLOC', 'ULT@MIDAS_6261_U2', 'IB@6261[CLASSHOT]',
       'FB@6261[CLASSHOT]','ARR_SPECKLE_SCREEN::PHYTON_X_USERFUNC_K_END_X_X_X_X_PBISTSPECKLE_INDICATOR::DUTCPUBAD_1@6261[CLASSHOT]',
       'TEST RESULTS', 'TEST RESULTS BITS', 'TR_BITS', 'INCOMING',
       'INCOMING BITS', 'INC_BITS', 'OUTGOING', 'OUTGOING BITS', 'OUT_BITS','DELTA']
rowna_df = check_rowna(df,supporting_fs)
rowna_df

,SPECKLE,VID,Total Null,% of Null
8213,0,M1GC346401871,1593,100.00
10375,0,M11H9B4300835,1593,100.00
10377,0,M11H9B4301228,1593,100.00
10378,0,M11H9B4300032,1593,100.00
10379,0,M11H9B4301227,1593,100.00
...,...,...,...,...
11806,0,M05C0K5600161,1,0.06
11811,0,M0U25B1100396,1,0.06
11814,0,M0U25B1100907,1,0.06
11815,0,M0U25B1100798,1,0.06


In [13]:
#get rows with 100% NA
rowallNA_df = rowna_df.loc[rowna_df["% of Null"] ==100.00]
rowallNA_df

,SPECKLE,VID,Total Null,% of Null
8213,0,M1GC346401871,1593,100.0
10375,0,M11H9B4300835,1593,100.0
10377,0,M11H9B4301228,1593,100.0
10378,0,M11H9B4300032,1593,100.0
10379,0,M11H9B4301227,1593,100.0
...,...,...,...,...
5463,0,M10P67G000951,1593,100.0
5464,0,M10P67G001238,1593,100.0
5468,0,M17U1C0300567,1593,100.0
5465,0,M10P67G000323,1593,100.0


In [14]:
#drop rows with 100% NA
df = df[~df["VID"].isin(rowallNA_df["VID"])]
df = df.reset_index(drop=True)
df

,SPECKLE,VID,ULT@MIDAS_6261_U1,SORTLOT,SORTLOT7,WAFER,XLOC,YLOC,ULT@MIDAS_6261_U2,IB@6261[CLASSHOT],...,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDCLK1,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDDATA0,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDDATA1,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TAP_DIS_STRAP_N,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TAP_ODT_EN,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_AGENT,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_PLTEN,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSCL,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSDA,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VSENSEPMAX
0,1,M10P67G000001,N0286920_293_6_-3,N0286920,N028692,293,6,-3,H0280460_443_2_1,1,...,-8.210000e-07,-8.210000e-07,-8.330000e-07,-2.100000e-07,-2.240000e-07,-1.950000e-07,-1.910000e-07,-8.990000e-07,-8.840000e-07,-1.970000e-07
1,1,M10P67G000962,N0286920_225_5_-5,N0286920,N028692,225,5,-5,H0280460_443_2_18,1,...,-8.670000e-07,-8.870000e-07,-8.740000e-07,-2.110000e-07,-2.100000e-07,-1.900000e-07,-2.280000e-07,-8.620000e-07,-8.610000e-07,-1.920000e-07
2,1,M10P67G000808,N0292860_135_6_0,N0292860,N029286,135,6,0,H0256240_667_-10_6,1,...,-6.000000e-07,-6.500000e-07,-6.380000e-07,-1.440000e-07,-1.380000e-07,-1.440000e-07,-1.620000e-07,-5.820000e-07,-6.150000e-07,-1.460000e-07
3,1,M17U1C0300573,N0293360_159_3_1,N0293360,N029336,159,3,1,H0280460_443_0_-14,1,...,-1.170000e-06,-1.220000e-06,-1.230000e-06,-2.950000e-07,-2.930000e-07,-3.190000e-07,-3.070000e-07,-1.220000e-06,-1.190000e-06,-2.960000e-07
4,1,M10P67G000433,N0293340_045_1_5,N0293340,N029334,45,1,5,H0280460_444_9_13,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6372,0,M1G88H4701199,N0326460_212_4_6,N0326460,N032646,212,4,6,H0310990_073_5_-2,1,...,-7.190000e-07,-7.620000e-07,-7.720000e-07,-1.810000e-07,-1.860000e-07,-1.720000e-07,-1.780000e-07,-7.080000e-07,-7.430000e-07,-1.720000e-07
6373,0,M1G88H4700911,N0326460_209_-1_-2,N0326460,N032646,209,-1,-2,H0310990_073_-3_18,1,...,-9.140000e-07,-9.750000e-07,-9.730000e-07,-2.420000e-07,-2.400000e-07,-2.330000e-07,-2.310000e-07,-8.940000e-07,-8.840000e-07,-2.310000e-07
6374,0,M1G88H4700361,N0370830_775_-3_0,N0370830,N037083,775,-3,0,H0310990_083_-14_-3,1,...,-1.010000e-06,-9.770000e-07,-1.000000e-06,-2.500000e-07,-2.510000e-07,-3.010000e-07,-2.310000e-07,-9.300000e-07,-9.430000e-07,-2.310000e-07
6375,0,M1G88H4700053,N0342690_197_1_-6,N0342690,N034269,197,1,-6,H0310990_073_-10_15,14,...,-1.060000e-06,-1.120000e-06,-1.130000e-06,-2.610000e-07,-2.700000e-07,-2.700000e-07,-2.720000e-07,-1.090000e-06,-1.110000e-06,-2.270000e-07


In [15]:
df["SPECKLE"].value_counts()

0    4608
1    1769
Name: SPECKLE, dtype: int64

In [16]:
#check row na
supporting_fs = ['ULT@MIDAS_6261_U1', 'SORTLOT', 'SORTLOT7', 'WAFER',
       'XLOC', 'YLOC', 'ULT@MIDAS_6261_U2', 'IB@6261[CLASSHOT]',
       'FB@6261[CLASSHOT]','ARR_SPECKLE_SCREEN::PHYTON_X_USERFUNC_K_END_X_X_X_X_PBISTSPECKLE_INDICATOR::DUTCPUBAD_1@6261[CLASSHOT]',
       'TEST RESULTS', 'TEST RESULTS BITS', 'TR_BITS', 'INCOMING',
       'INCOMING BITS', 'INC_BITS', 'OUTGOING', 'OUTGOING BITS', 'OUT_BITS','DELTA']
check_rowna(df,supporting_fs)

,SPECKLE,VID,Total Null,% of Null
573,1,M1888SS100613,1310,82.23
579,1,M1888SS100734,1310,82.23
581,1,M1888SS100614,1310,82.23
582,1,M1888SS100616,1310,82.23
577,1,M1888SS100692,1310,82.23
...,...,...,...,...
1536,0,M0LN682800123,1,0.06
1535,0,M0LN682801353,1,0.06
1534,0,M0LN682801352,1,0.06
1533,0,M0LN682800122,1,0.06


In [17]:
df = df.drop(['PTH_POWER::POWER_X_SCREEN_E_BEGIN_X_X_X_X_CALC_PP_CDYN_INDICATOR_CDYN_DATA@132110'],axis=1)

In [18]:
df.shape

(6377, 1614)

In [19]:
#NA imputation
def NA_impute(df,imptype):
    """
    Impute NA
    df[DataFrame]:df
    imptype[string]: "mean" to impute data with mean, "median" to impute data with median
    """
    if imptype == "mean":        
        df = df.fillna(df.mean())
    if imptype == "median":
        df = df.fillna(df.median())
    return df

In [20]:
df = NA_impute(df,imptype="median")
df

,SPECKLE,VID,ULT@MIDAS_6261_U1,SORTLOT,SORTLOT7,WAFER,XLOC,YLOC,ULT@MIDAS_6261_U2,IB@6261[CLASSHOT],...,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDCLK1,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDDATA0,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDDATA1,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TAP_DIS_STRAP_N,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TAP_ODT_EN,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_AGENT,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_PLTEN,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSCL,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSDA,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VSENSEPMAX
0,1,M10P67G000001,N0286920_293_6_-3,N0286920,N028692,293,6,-3,H0280460_443_2_1,1,...,-8.210000e-07,-8.210000e-07,-8.330000e-07,-2.100000e-07,-2.240000e-07,-1.950000e-07,-1.910000e-07,-8.990000e-07,-8.840000e-07,-1.970000e-07
1,1,M10P67G000962,N0286920_225_5_-5,N0286920,N028692,225,5,-5,H0280460_443_2_18,1,...,-8.670000e-07,-8.870000e-07,-8.740000e-07,-2.110000e-07,-2.100000e-07,-1.900000e-07,-2.280000e-07,-8.620000e-07,-8.610000e-07,-1.920000e-07
2,1,M10P67G000808,N0292860_135_6_0,N0292860,N029286,135,6,0,H0256240_667_-10_6,1,...,-6.000000e-07,-6.500000e-07,-6.380000e-07,-1.440000e-07,-1.380000e-07,-1.440000e-07,-1.620000e-07,-5.820000e-07,-6.150000e-07,-1.460000e-07
3,1,M17U1C0300573,N0293360_159_3_1,N0293360,N029336,159,3,1,H0280460_443_0_-14,1,...,-1.170000e-06,-1.220000e-06,-1.230000e-06,-2.950000e-07,-2.930000e-07,-3.190000e-07,-3.070000e-07,-1.220000e-06,-1.190000e-06,-2.960000e-07
4,1,M10P67G000433,N0293340_045_1_5,N0293340,N029334,45,1,5,H0280460_444_9_13,1,...,-8.520000e-07,-8.570000e-07,-8.640000e-07,-2.150000e-07,-2.250000e-07,-2.190000e-07,-2.170000e-07,-8.545000e-07,-8.510000e-07,-2.130000e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6372,0,M1G88H4701199,N0326460_212_4_6,N0326460,N032646,212,4,6,H0310990_073_5_-2,1,...,-7.190000e-07,-7.620000e-07,-7.720000e-07,-1.810000e-07,-1.860000e-07,-1.720000e-07,-1.780000e-07,-7.080000e-07,-7.430000e-07,-1.720000e-07
6373,0,M1G88H4700911,N0326460_209_-1_-2,N0326460,N032646,209,-1,-2,H0310990_073_-3_18,1,...,-9.140000e-07,-9.750000e-07,-9.730000e-07,-2.420000e-07,-2.400000e-07,-2.330000e-07,-2.310000e-07,-8.940000e-07,-8.840000e-07,-2.310000e-07
6374,0,M1G88H4700361,N0370830_775_-3_0,N0370830,N037083,775,-3,0,H0310990_083_-14_-3,1,...,-1.010000e-06,-9.770000e-07,-1.000000e-06,-2.500000e-07,-2.510000e-07,-3.010000e-07,-2.310000e-07,-9.300000e-07,-9.430000e-07,-2.310000e-07
6375,0,M1G88H4700053,N0342690_197_1_-6,N0342690,N034269,197,1,-6,H0310990_073_-10_15,14,...,-1.060000e-06,-1.120000e-06,-1.130000e-06,-2.610000e-07,-2.700000e-07,-2.700000e-07,-2.720000e-07,-1.090000e-06,-1.110000e-06,-2.270000e-07


In [21]:
check_rowna(df,supporting_fs)

,SPECKLE,VID,Total Null,% of Null


### 3) Drop supporting features

In [22]:
supporting_fs = ['VID','ULT@MIDAS_6261_U1', 'SORTLOT', 'SORTLOT7', 'WAFER',
       'XLOC', 'YLOC', 'ULT@MIDAS_6261_U2', 'IB@6261[CLASSHOT]',
       'FB@6261[CLASSHOT]','ARR_SPECKLE_SCREEN::PHYTON_X_USERFUNC_K_END_X_X_X_X_PBISTSPECKLE_INDICATOR::DUTCPUBAD_1@6261[CLASSHOT]',
       'TEST RESULTS', 'TEST RESULTS BITS', 'TR_BITS', 'INCOMING',
       'INCOMING BITS', 'INC_BITS', 'OUTGOING', 'OUTGOING BITS', 'OUT_BITS','DELTA']
df_sf = df[supporting_fs] #keep supporting fs
df = df.drop(supporting_fs,axis=1)
df

,SPECKLE,IDV_0001_SVT3GNES12_FULLDIE_SOC_0650_MED@132110,IDV_0001_SVT3GNES12_FULLDIE_SOC_0950_MED@132110,IDV_0001_SVT3GNES12_FULLDIE_SOC_TALL_0650_MED@132110,IDV_0001_SVT3GNES12_FULLDIE_SOC_TALL_0950_MED@132110,IDV_0002_SVT3GNES12_FULLDIE_SOC_0650_MED@132110,IDV_0002_SVT3GNES12_FULLDIE_SOC_0950_MED@132110,IDV_0002_SVT3GNES12_FULLDIE_SOC_TALL_0650_MED@132110,IDV_0002_SVT3GNES12_FULLDIE_SOC_TALL_0950_MED@132110,IDV_0003_SVT3GSEM12_FULLDIE_SOC_0650_MED@132110,...,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDCLK1,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDDATA0,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDDATA1,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TAP_DIS_STRAP_N,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TAP_ODT_EN,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_AGENT,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_PLTEN,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSCL,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSDA,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VSENSEPMAX
0,1,6016.90,12304.2,5791.55,12112.7,5994.37,12293.0,5887.32,12214.1,5245.07,...,-8.210000e-07,-8.210000e-07,-8.330000e-07,-2.100000e-07,-2.240000e-07,-1.950000e-07,-1.910000e-07,-8.990000e-07,-8.840000e-07,-1.970000e-07
1,1,6073.24,12214.1,6005.63,12163.4,6078.87,12174.7,6061.97,12236.6,5295.77,...,-8.670000e-07,-8.870000e-07,-8.740000e-07,-2.110000e-07,-2.100000e-07,-1.900000e-07,-2.280000e-07,-8.620000e-07,-8.610000e-07,-1.920000e-07
2,1,6343.66,12231.0,6163.38,12028.2,6264.79,12152.1,6219.72,12129.6,5695.77,...,-6.000000e-07,-6.500000e-07,-6.380000e-07,-1.440000e-07,-1.380000e-07,-1.440000e-07,-1.620000e-07,-5.820000e-07,-6.150000e-07,-1.460000e-07
3,1,5566.20,11453.5,5442.25,11357.8,5554.93,11414.1,5476.06,11391.5,4811.27,...,-1.170000e-06,-1.220000e-06,-1.230000e-06,-2.950000e-07,-2.930000e-07,-3.190000e-07,-3.070000e-07,-1.220000e-06,-1.190000e-06,-2.960000e-07
4,1,6247.89,12293.0,6112.68,12180.3,6219.72,12270.4,6163.38,12259.2,5470.42,...,-8.520000e-07,-8.570000e-07,-8.640000e-07,-2.150000e-07,-2.250000e-07,-2.190000e-07,-2.170000e-07,-8.545000e-07,-8.510000e-07,-2.130000e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6372,0,6664.79,12450.7,6602.82,12371.8,6670.42,12462.0,6636.62,12416.9,5954.93,...,-7.190000e-07,-7.620000e-07,-7.720000e-07,-1.810000e-07,-1.860000e-07,-1.720000e-07,-1.780000e-07,-7.080000e-07,-7.430000e-07,-1.720000e-07
6373,0,6208.45,12056.3,6095.77,12000.0,6163.38,12056.3,6185.92,12078.9,5453.52,...,-9.140000e-07,-9.750000e-07,-9.730000e-07,-2.420000e-07,-2.400000e-07,-2.330000e-07,-2.310000e-07,-8.940000e-07,-8.840000e-07,-2.310000e-07
6374,0,6202.82,12169.0,6163.38,12118.3,6253.52,12185.9,6264.79,12247.9,5526.76,...,-1.010000e-06,-9.770000e-07,-1.000000e-06,-2.500000e-07,-2.510000e-07,-3.010000e-07,-2.310000e-07,-9.300000e-07,-9.430000e-07,-2.310000e-07
6375,0,5526.76,11549.3,5397.18,11493.0,5464.79,11509.9,5459.15,11554.9,4867.61,...,-1.060000e-06,-1.120000e-06,-1.130000e-06,-2.610000e-07,-2.700000e-07,-2.700000e-07,-2.720000e-07,-1.090000e-06,-1.110000e-06,-2.270000e-07


### 4) Converting negative to positive values

In [23]:
#To create a dataframe df_keep with IDV and HVQK token family columns
cols_to_keep=["IDV", "HVQK"]
df_keep = df[df.columns[df.columns.str.startswith(tuple(cols_to_keep))]]

In [24]:
df_keep.head()

,IDV_0001_SVT3GNES12_FULLDIE_SOC_0650_MED@132110,IDV_0001_SVT3GNES12_FULLDIE_SOC_0950_MED@132110,IDV_0001_SVT3GNES12_FULLDIE_SOC_TALL_0650_MED@132110,IDV_0001_SVT3GNES12_FULLDIE_SOC_TALL_0950_MED@132110,IDV_0002_SVT3GNES12_FULLDIE_SOC_0650_MED@132110,IDV_0002_SVT3GNES12_FULLDIE_SOC_0950_MED@132110,IDV_0002_SVT3GNES12_FULLDIE_SOC_TALL_0650_MED@132110,IDV_0002_SVT3GNES12_FULLDIE_SOC_TALL_0950_MED@132110,IDV_0003_SVT3GSEM12_FULLDIE_SOC_0650_MED@132110,IDV_0003_SVT3GSEM12_FULLDIE_SOC_0950_MED@132110,...,HVQK_VMIN_SHIFT_SCAN_UNCORE_LLCSFP02@132110,HVQK_VMIN_SHIFT_SCAN_UNCORE_LLCSFP11@132110,HVQK_VMIN_SHIFT_SCAN_UNCORE_LLCSFP12@132110,HVQK_VMIN_SHIFT_SCAN_UNCORE_LLCSFP21@132110,HVQK_VMIN_SHIFT_SCAN_UNCORE_LLCSFP23@132110,HVQK_VMIN_SHIFT_SCAN_UNCORE_MISC@132110,HVQK_VMIN_SHIFT_SCAN_UNCORE_PCIE@132110,HVQK_VMIN_SHIFT_SCAN_UNCORE_RLINK@132110,HVQK_VMIN_SHIFT_SCAN_UNCORE_SA@132110,HVQK_VMIN_SHIFT_SCAN_UNCORE_VNN@132110
0,6016.90,12304.2,5791.55,12112.7,5994.37,12293.0,5887.32,12214.1,5245.07,11600.0,...,0.0,0.00,0.02,0.00,0.0,0.00,0.0,0.0,0.0,0.0
1,6073.24,12214.1,6005.63,12163.4,6078.87,12174.7,6061.97,12236.6,5295.77,11560.6,...,0.0,0.02,-555.00,0.02,0.0,0.00,0.0,0.0,0.0,0.0
2,6343.66,12231.0,6163.38,12028.2,6264.79,12152.1,6219.72,12129.6,5695.77,11639.4,...,0.0,0.00,0.00,-555.00,0.0,0.02,0.0,0.0,0.0,0.0
3,5566.20,11453.5,5442.25,11357.8,5554.93,11414.1,5476.06,11391.5,4811.27,10777.5,...,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0
4,6247.89,12293.0,6112.68,12180.3,6219.72,12270.4,6163.38,12259.2,5470.42,11639.4,...,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0


In [25]:
#To create dataframe - drop columns with IDV token family columns
df_to_convert = df.drop([col for col in df if col.startswith('IDV')], axis=1)

In [26]:
#To create dataframe - drop columns with HVQK token family columns
df_to_convert = df_to_convert.drop([col for col in df if col.startswith('HVQK')], axis=1)

In [27]:
df_to_convert.head()

,SPECKLE,TPI_SIU_STATIC::SIU_STATIC_AM_K_START_X_X_X_SNR_VNOM_VCCINF_1P05_SIUP_SICC_VCCADLLOPI_V1@132110,TPI_SIU_STATIC::SIU_STATIC_AM_K_START_X_X_X_SNR_VNOM_VCCINF_1P05_SIUP_SICC_VCCADLLOPI_V1@132150,TPI_SIU_STATIC::SIU_STATIC_AM_K_START_X_X_X_SNR_VNOM_VCCINF_1P05_SIUP_SICC_VCCAGSH_CLM_EHV_V1@132110,TPI_SIU_STATIC::SIU_STATIC_AM_K_START_X_X_X_SNR_VNOM_VCCINF_1P05_SIUP_SICC_VCCAGSH_CLM_EHV_V1@132150,TPI_SIU_STATIC::SIU_STATIC_AM_K_START_X_X_X_SNR_VNOM_VCCINF_1P05_SIUP_SICC_VCCAOPI_V1@132110,TPI_SIU_STATIC::SIU_STATIC_AM_K_START_X_X_X_SNR_VNOM_VCCINF_1P05_SIUP_SICC_VCCAOPI_V1@132150,TPI_SIU_STATIC::SIU_STATIC_AM_K_START_X_X_X_SNR_VNOM_VCCINF_1P05_SIUP_SICC_VCCAPLLOPI_FAD_DIGICKSI0_V1@132110,TPI_SIU_STATIC::SIU_STATIC_AM_K_START_X_X_X_SNR_VNOM_VCCINF_1P05_SIUP_SICC_VCCAPLLOPI_FAD_DIGICKSI0_V1@132150,TPI_SIU_STATIC::SIU_STATIC_AM_K_START_X_X_X_SNR_VNOM_VCCINF_1P05_SIUP_SICC_VCCCORE_M01_V1@132110,...,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDCLK1,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDDATA0,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDDATA1,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TAP_DIS_STRAP_N,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TAP_ODT_EN,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_AGENT,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_PLTEN,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSCL,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSDA,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VSENSEPMAX
0,1,0.044625,0.041133,0.020178,0.021403,0.013161,0.050175,0.002702,0.013882,0.062004,...,-8.210000e-07,-8.210000e-07,-8.330000e-07,-2.100000e-07,-2.240000e-07,-1.950000e-07,-1.910000e-07,-8.990000e-07,-8.840000e-07,-1.970000e-07
1,1,0.037500,0.054445,0.020281,0.021683,0.019436,0.092677,0.002525,0.013907,0.080011,...,-8.670000e-07,-8.870000e-07,-8.740000e-07,-2.110000e-07,-2.100000e-07,-1.900000e-07,-2.280000e-07,-8.620000e-07,-8.610000e-07,-1.920000e-07
2,1,0.047672,0.049816,0.021016,0.022506,0.013063,0.060671,0.007954,0.013618,0.076174,...,-6.000000e-07,-6.500000e-07,-6.380000e-07,-1.440000e-07,-1.380000e-07,-1.440000e-07,-1.620000e-07,-5.820000e-07,-6.150000e-07,-1.460000e-07
3,1,0.047462,0.049402,0.018823,0.019772,0.010239,0.043242,0.001651,0.007795,0.031176,...,-1.170000e-06,-1.220000e-06,-1.230000e-06,-2.950000e-07,-2.930000e-07,-3.190000e-07,-3.070000e-07,-1.220000e-06,-1.190000e-06,-2.960000e-07
4,1,0.048798,0.050560,0.019342,0.021034,0.013236,0.066157,0.002411,0.012175,0.048849,...,-8.520000e-07,-8.570000e-07,-8.640000e-07,-2.150000e-07,-2.250000e-07,-2.190000e-07,-2.170000e-07,-8.545000e-07,-8.510000e-07,-2.130000e-07


In [28]:
#Convert negative columns with specific token family prefix
df_positive = df_to_convert.abs()

In [29]:
df_positive.head()

,SPECKLE,TPI_SIU_STATIC::SIU_STATIC_AM_K_START_X_X_X_SNR_VNOM_VCCINF_1P05_SIUP_SICC_VCCADLLOPI_V1@132110,TPI_SIU_STATIC::SIU_STATIC_AM_K_START_X_X_X_SNR_VNOM_VCCINF_1P05_SIUP_SICC_VCCADLLOPI_V1@132150,TPI_SIU_STATIC::SIU_STATIC_AM_K_START_X_X_X_SNR_VNOM_VCCINF_1P05_SIUP_SICC_VCCAGSH_CLM_EHV_V1@132110,TPI_SIU_STATIC::SIU_STATIC_AM_K_START_X_X_X_SNR_VNOM_VCCINF_1P05_SIUP_SICC_VCCAGSH_CLM_EHV_V1@132150,TPI_SIU_STATIC::SIU_STATIC_AM_K_START_X_X_X_SNR_VNOM_VCCINF_1P05_SIUP_SICC_VCCAOPI_V1@132110,TPI_SIU_STATIC::SIU_STATIC_AM_K_START_X_X_X_SNR_VNOM_VCCINF_1P05_SIUP_SICC_VCCAOPI_V1@132150,TPI_SIU_STATIC::SIU_STATIC_AM_K_START_X_X_X_SNR_VNOM_VCCINF_1P05_SIUP_SICC_VCCAPLLOPI_FAD_DIGICKSI0_V1@132110,TPI_SIU_STATIC::SIU_STATIC_AM_K_START_X_X_X_SNR_VNOM_VCCINF_1P05_SIUP_SICC_VCCAPLLOPI_FAD_DIGICKSI0_V1@132150,TPI_SIU_STATIC::SIU_STATIC_AM_K_START_X_X_X_SNR_VNOM_VCCINF_1P05_SIUP_SICC_VCCCORE_M01_V1@132110,...,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDCLK1,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDDATA0,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDDATA1,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TAP_DIS_STRAP_N,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TAP_ODT_EN,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_AGENT,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_PLTEN,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSCL,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSDA,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VSENSEPMAX
0,1,0.044625,0.041133,0.020178,0.021403,0.013161,0.050175,0.002702,0.013882,0.062004,...,8.210000e-07,8.210000e-07,8.330000e-07,2.100000e-07,2.240000e-07,1.950000e-07,1.910000e-07,8.990000e-07,8.840000e-07,1.970000e-07
1,1,0.037500,0.054445,0.020281,0.021683,0.019436,0.092677,0.002525,0.013907,0.080011,...,8.670000e-07,8.870000e-07,8.740000e-07,2.110000e-07,2.100000e-07,1.900000e-07,2.280000e-07,8.620000e-07,8.610000e-07,1.920000e-07
2,1,0.047672,0.049816,0.021016,0.022506,0.013063,0.060671,0.007954,0.013618,0.076174,...,6.000000e-07,6.500000e-07,6.380000e-07,1.440000e-07,1.380000e-07,1.440000e-07,1.620000e-07,5.820000e-07,6.150000e-07,1.460000e-07
3,1,0.047462,0.049402,0.018823,0.019772,0.010239,0.043242,0.001651,0.007795,0.031176,...,1.170000e-06,1.220000e-06,1.230000e-06,2.950000e-07,2.930000e-07,3.190000e-07,3.070000e-07,1.220000e-06,1.190000e-06,2.960000e-07
4,1,0.048798,0.050560,0.019342,0.021034,0.013236,0.066157,0.002411,0.012175,0.048849,...,8.520000e-07,8.570000e-07,8.640000e-07,2.150000e-07,2.250000e-07,2.190000e-07,2.170000e-07,8.545000e-07,8.510000e-07,2.130000e-07


In [30]:
#To create dataset with some parameters converted to absolute values
#Concatenating df_keep and df_positive along columns
df = pd.concat([df_keep, df_positive], axis=1)

In [31]:
df.head()

,IDV_0001_SVT3GNES12_FULLDIE_SOC_0650_MED@132110,IDV_0001_SVT3GNES12_FULLDIE_SOC_0950_MED@132110,IDV_0001_SVT3GNES12_FULLDIE_SOC_TALL_0650_MED@132110,IDV_0001_SVT3GNES12_FULLDIE_SOC_TALL_0950_MED@132110,IDV_0002_SVT3GNES12_FULLDIE_SOC_0650_MED@132110,IDV_0002_SVT3GNES12_FULLDIE_SOC_0950_MED@132110,IDV_0002_SVT3GNES12_FULLDIE_SOC_TALL_0650_MED@132110,IDV_0002_SVT3GNES12_FULLDIE_SOC_TALL_0950_MED@132110,IDV_0003_SVT3GSEM12_FULLDIE_SOC_0650_MED@132110,IDV_0003_SVT3GSEM12_FULLDIE_SOC_0950_MED@132110,...,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDCLK1,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDDATA0,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDDATA1,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TAP_DIS_STRAP_N,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TAP_ODT_EN,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_AGENT,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_PLTEN,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSCL,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSDA,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VSENSEPMAX
0,6016.90,12304.2,5791.55,12112.7,5994.37,12293.0,5887.32,12214.1,5245.07,11600.0,...,8.210000e-07,8.210000e-07,8.330000e-07,2.100000e-07,2.240000e-07,1.950000e-07,1.910000e-07,8.990000e-07,8.840000e-07,1.970000e-07
1,6073.24,12214.1,6005.63,12163.4,6078.87,12174.7,6061.97,12236.6,5295.77,11560.6,...,8.670000e-07,8.870000e-07,8.740000e-07,2.110000e-07,2.100000e-07,1.900000e-07,2.280000e-07,8.620000e-07,8.610000e-07,1.920000e-07
2,6343.66,12231.0,6163.38,12028.2,6264.79,12152.1,6219.72,12129.6,5695.77,11639.4,...,6.000000e-07,6.500000e-07,6.380000e-07,1.440000e-07,1.380000e-07,1.440000e-07,1.620000e-07,5.820000e-07,6.150000e-07,1.460000e-07
3,5566.20,11453.5,5442.25,11357.8,5554.93,11414.1,5476.06,11391.5,4811.27,10777.5,...,1.170000e-06,1.220000e-06,1.230000e-06,2.950000e-07,2.930000e-07,3.190000e-07,3.070000e-07,1.220000e-06,1.190000e-06,2.960000e-07
4,6247.89,12293.0,6112.68,12180.3,6219.72,12270.4,6163.38,12259.2,5470.42,11639.4,...,8.520000e-07,8.570000e-07,8.640000e-07,2.150000e-07,2.250000e-07,2.190000e-07,2.170000e-07,8.545000e-07,8.510000e-07,2.130000e-07


### 5) Handling unary columns 

In [32]:
def unary(df):
    """
    Checks for unary columns. If there are unary columns, the unary columns will be printed and removed from the df
    df[DataFrame]: input dataframe
    """
    unarycolumns = [col for col in df.columns if len(df[col].unique())==1]
    if unarycolumns:
        print("The unary column are:",unarycolumns)
        df = df.drop(unarycolumns,axis=1)
        print("Unary columns dropped!")
        return df 
    else:
        print("There are no unary columns!")

In [33]:
unary(df)

There are no unary columns!


In [ ]:
# supporting_fs = ['SPECKLE','VID','ULT@MIDAS_6261_U1', 'SORTLOT', 'SORTLOT7', 'WAFER',
#        'XLOC', 'YLOC', 'ULT@MIDAS_6261_U2', 'IB@6261[CLASSHOT]',
#        'FB@6261[CLASSHOT]','ARR_SPECKLE_SCREEN::PHYTON_X_USERFUNC_K_END_X_X_X_X_PBISTSPECKLE_INDICATOR::DUTCPUBAD_1@6261[CLASSHOT]',
#        'TEST RESULTS', 'TEST RESULTS BITS', 'TR_BITS', 'INCOMING',
#        'INCOMING BITS', 'INC_BITS', 'OUTGOING', 'OUTGOING BITS', 'OUT_BITS','DELTA']
# sf_df = df[supporting_fs]
# df1 = df.drop(supporting_fs,axis=1) 
# df1.head()

In [ ]:
# df1.columns.difference(df2.columns)

In [ ]:
# df1['PTH_POWER::POWER_X_SCREEN_E_BEGIN_X_X_X_X_CALC_PP_CDYN_INDICATOR_CDYN_DATA@132110']

In [ ]:
# df2 = df1.drop(['PTH_POWER::POWER_X_SCREEN_E_BEGIN_X_X_X_X_CALC_PP_CDYN_INDICATOR_CDYN_DATA@132110'],axis=1)
# df_neg = df2.loc[:,((df2 == -555).any()) | ((df2 == -999).any())]

In [ ]:
# df_neg_sf = pd.concat([sf_df,df_neg],axis=1)
# df_neg_sf

In [ ]:
# df_neg_sf.to_csv(path+"/DataPreparation/Negative_Cols_555_999.csv",index=False)

### 6) Train-validation split

In [34]:
from sklearn.model_selection import train_test_split
from collections import Counter
def randomsamp(df,val_size):
    """
    Split whole dataset into train and validation using random sampling
    Returns X_train, X_val, y_train, y_val
    df[DataFrame]: input dataframe    
    val_size[float]:Should be between 0.0 and 1.0 and represent the proportion of the dataset to include in the val split. 
                    Train size is complement of val size
    """
    X= df.drop(["SPECKLE"],axis=1)
    y= df["SPECKLE"] 
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = val_size,random_state=42,stratify= df["SPECKLE"])
    if type(val_size)==float:
        print("Train-val split completed with",(1-val_size)*100,"-",val_size*100,"split in train-val")
    print("Shape of X_train is:", X_train.shape)
    print("Shape of X_val is:",X_val.shape)
    print("Shape of y_train is:",y_train.shape)
    print("Shape of y_val is:",y_val.shape)
    print("Distribution of y_train:",Counter(y_train))
    print("Distribution of y_val:",Counter(y_val))
    
    X_train = X_train.reset_index(drop=True)
    X_val = X_val.reset_index(drop=True)
    y_train = y_train.reset_index(drop=True)
    y_val = y_val.reset_index(drop=True)        
           
    return X_train, X_val, y_train, y_val

def targetrandomsamp(df,speckle_test_size,nonspeckle_test_size):
    """
    Dataset is split into speckle/non-speckle first. The speckle/non-speckle datasets are then split 
    into train and validation using random sampling, followed by merging of the speckle/non-speckle  
    to return X_train, X_val, y_train, y_val
    
    df[DataFrame]: input dataframe  
    
    speckle_test_size[float/int]: Val size for speckle 
    -If float, should be between 0.0 and 1.0 and represent the proportion of the dataset to include in the val split. 
    -If int, represents the absolute number of val samples.
    -Train size is complement of val size
    
    nonspeckle_test_size[float/int]: Val size for non-speckle 
    -If float, should be between 0.0 and 1.0 and represent the proportion of the dataset to include in the val split. 
    -If int, represents the absolute number of val samples.
    -Train size is complement of val size
    
    """
    #split data into speckle/nonspeckle
    df_s = df.loc[df["SPECKLE"] == 1] #speckle
    df_ns = df.loc[df["SPECKLE"] == 0] #nonspeckle
    
    #split speckle/nonspeckle data into train/validation
    print("For speckle data:")
    X_train_s, X_val_s, y_train_s, y_val_s = randomsamp(df_s,val_size=speckle_test_size)
    print("\nFor non-speckle data:")
    X_train_ns, X_val_ns, y_train_ns, y_val_ns = randomsamp(df_ns,val_size=nonspeckle_test_size)
    
    #concat the speckle/non-speckle train and validation
    X_train = pd.concat([X_train_s,X_train_ns], ignore_index=True)
    X_val = pd.concat([X_val_s,X_val_ns], ignore_index=True)
    y_train = pd.concat([y_train_s,y_train_ns], ignore_index=True)
    y_val = pd.concat([y_val_s,y_val_ns], ignore_index=True)
    
    X_train = X_train.reset_index(drop=True)
    X_val = X_val.reset_index(drop=True)
    y_train = y_train.reset_index(drop=True)
    y_val = y_val.reset_index(drop=True)  
    
    print("\nFinal dataset:")
    print("Distribution of y_train:",Counter(y_train))
    print("Distribution of y_val:",Counter(y_val))
    return X_train, X_val, y_train, y_val

In [35]:
df["SPECKLE"].value_counts()

0    4608
1    1769
Name: SPECKLE, dtype: int64

In [36]:
#train-test split on whole data
X_train, X_val, y_train, y_val = randomsamp(df,val_size=0.3)

Train-val split completed with 70.0 - 30.0 split in train-val
Shape of X_train is: (4463, 1592)
Shape of X_val is: (1914, 1592)
Shape of y_train is: (4463,)
Shape of y_val is: (1914,)
Distribution of y_train: Counter({0: 3225, 1: 1238})
Distribution of y_val: Counter({0: 1383, 1: 531})


In [37]:
#train-test split on speckle/non-speckle then merge
X_train, X_val, y_train, y_val = targetrandomsamp(df,speckle_test_size=500,nonspeckle_test_size=2000)

For speckle data:
Shape of X_train is: (1269, 1592)
Shape of X_val is: (500, 1592)
Shape of y_train is: (1269,)
Shape of y_val is: (500,)
Distribution of y_train: Counter({1: 1269})
Distribution of y_val: Counter({1: 500})

For non-speckle data:
Shape of X_train is: (2608, 1592)
Shape of X_val is: (2000, 1592)
Shape of y_train is: (2608,)
Shape of y_val is: (2000,)
Distribution of y_train: Counter({0: 2608})
Distribution of y_val: Counter({0: 2000})

Final dataset:
Distribution of y_train: Counter({0: 2608, 1: 1269})
Distribution of y_val: Counter({0: 2000, 1: 500})


### 7) Scaling

In [38]:
#To scale X train and X validation (test) data using various scalers
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

def scale_data(X_train,X_val,scaler_type):
    """
    Scaling X train and validation with standardization or normalization
    params:
    X_train[DataFrame]: input X train
    X_val[DataFrame]: input X validation (test)
    scaler_type[None/string]: input scaling method
    - "Standardization" for Standard Scaler
    - "Normalization" for Min Max Scaler    

    """           
    if scaler_type == "Standardization":
        scaler = StandardScaler()
    if scaler_type == "Normalization":
        scaler = MinMaxScaler()

    X_train_scaled = scaler.fit_transform(X_train)
    X_train_scaled = pd.DataFrame(X_train_scaled,columns= X_train.columns)
    X_val_scaled = scaler.transform(X_val)
    X_val_scaled = pd.DataFrame(X_val_scaled,columns= X_val.columns)
       
    return X_train_scaled, X_val_scaled

In [43]:
#X train-test scaled using Standard Scaler
X_train_scaled, X_val_scaled = scale_data(X_train,X_val,scaler_type = "Standardization")

In [40]:
#X train-test scaled using Standard Scaler
X_train_scaled, X_val_scaled = scale_data(X_train,X_val,scaler_type = "Normalization")

In [44]:
X_train_scaled

,IDV_0001_SVT3GNES12_FULLDIE_SOC_0650_MED@132110,IDV_0001_SVT3GNES12_FULLDIE_SOC_0950_MED@132110,IDV_0001_SVT3GNES12_FULLDIE_SOC_TALL_0650_MED@132110,IDV_0001_SVT3GNES12_FULLDIE_SOC_TALL_0950_MED@132110,IDV_0002_SVT3GNES12_FULLDIE_SOC_0650_MED@132110,IDV_0002_SVT3GNES12_FULLDIE_SOC_0950_MED@132110,IDV_0002_SVT3GNES12_FULLDIE_SOC_TALL_0650_MED@132110,IDV_0002_SVT3GNES12_FULLDIE_SOC_TALL_0950_MED@132110,IDV_0003_SVT3GSEM12_FULLDIE_SOC_0650_MED@132110,IDV_0003_SVT3GSEM12_FULLDIE_SOC_0950_MED@132110,...,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDCLK1,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDDATA0,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDDATA1,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TAP_DIS_STRAP_N,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TAP_ODT_EN,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_AGENT,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_PLTEN,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSCL,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSDA,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VSENSEPMAX
0,-0.041617,-0.034975,-0.045543,-0.020180,-0.020361,-0.042600,-0.039771,-0.026008,-0.027624,-0.028717,...,0.841446,0.889031,0.849580,0.656445,0.920392,0.585307,0.979461,0.981362,1.191699,0.589730
1,0.838359,0.744007,0.802551,0.689176,0.849516,0.610374,0.929931,0.824323,0.999564,1.069016,...,1.413705,1.499352,1.323092,1.483460,1.177175,1.089165,0.858089,1.294833,1.281583,1.331503
2,-0.851211,-0.713969,-0.787633,-0.790719,-0.664068,-0.636213,-0.797889,-0.815267,-0.798015,-0.727468,...,0.665366,0.470526,0.634348,0.294626,0.695707,0.534921,0.702039,0.524591,0.607451,1.228000
3,0.204767,-0.174674,0.166465,-0.283984,0.118818,-0.161323,0.154163,-0.288975,0.100774,-0.228563,...,-0.100581,0.052021,-0.002740,-0.049964,-0.186985,-0.002528,0.389939,0.063341,0.059156,-0.100291
4,-0.780798,0.065013,-0.822978,0.040642,-0.872822,-0.101961,-0.780270,0.075299,-0.765937,-0.008874,...,1.677825,1.760917,1.753557,2.017574,1.257420,1.559432,1.343579,1.384396,1.371468,1.728265
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3872,1.102330,1.163459,1.032247,1.074625,1.301856,1.303039,0.912312,0.965865,1.272428,1.568276,...,-0.100581,-0.083122,-0.239496,-0.360094,-0.219083,-0.372024,-0.164907,-0.106829,-0.044211,-0.341798
3873,0.275148,-0.274661,0.201810,-0.993340,0.205809,-0.319736,0.171814,-0.997404,1.063777,0.011323,...,-1.980232,-1.839972,-2.004402,-2.232939,-1.615341,-2.068346,-2.151946,-1.951828,-2.008184,-1.877094
3874,1.243125,0.384477,1.332633,0.608199,1.354045,0.551013,1.194404,0.480167,0.999564,0.210814,...,-0.985381,-0.802427,-1.010028,-0.825291,-0.844992,-0.993449,-0.875802,-0.697945,-0.965525,-0.945566
3875,0.222354,0.524531,0.131151,0.324241,0.223226,0.353259,0.242321,0.358742,0.148920,0.250854,...,0.498091,0.470526,0.634348,0.570298,0.872245,0.686078,0.736717,0.578328,0.373751,0.727734


In [45]:
X_val_scaled

,IDV_0001_SVT3GNES12_FULLDIE_SOC_0650_MED@132110,IDV_0001_SVT3GNES12_FULLDIE_SOC_0950_MED@132110,IDV_0001_SVT3GNES12_FULLDIE_SOC_TALL_0650_MED@132110,IDV_0001_SVT3GNES12_FULLDIE_SOC_TALL_0950_MED@132110,IDV_0002_SVT3GNES12_FULLDIE_SOC_0650_MED@132110,IDV_0002_SVT3GNES12_FULLDIE_SOC_0950_MED@132110,IDV_0002_SVT3GNES12_FULLDIE_SOC_TALL_0650_MED@132110,IDV_0002_SVT3GNES12_FULLDIE_SOC_TALL_0950_MED@132110,IDV_0003_SVT3GSEM12_FULLDIE_SOC_0650_MED@132110,IDV_0003_SVT3GSEM12_FULLDIE_SOC_0950_MED@132110,...,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDCLK1,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDDATA0,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDDATA1,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TAP_DIS_STRAP_N,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TAP_ODT_EN,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_AGENT,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_PLTEN,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSCL,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSDA,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VSENSEPMAX
0,0.803153,1.223380,0.519853,1.013802,0.623346,1.085264,0.542063,0.965865,0.614368,1.129253,...,-1.619268,-1.530452,-1.552414,-1.411093,-1.470901,-1.732440,-1.447986,-1.409971,-1.540785,-1.394080
1,0.503944,0.724151,0.237155,0.486554,0.605961,0.749118,0.330479,0.560997,0.004482,0.450345,...,0.273589,0.113053,0.203883,0.742593,0.503119,0.450945,0.095177,0.170817,0.297350,0.244720
2,-0.041617,-0.034975,-0.045543,-0.020180,-0.020361,-0.042600,-0.039771,-0.026008,-0.027624,-0.028717,...,-0.104983,-0.157232,-0.037177,-0.342865,-0.186985,-0.187276,-0.164907,-0.071004,0.072639,-0.341798
3,-0.745623,-0.713969,-0.787633,-0.689228,-0.698840,-0.715244,-0.762651,-0.774673,-1.343716,-1.026882,...,0.973506,1.237786,0.935673,1.276707,1.193224,1.021984,1.135512,1.250051,1.011930,0.848488
4,-0.886417,0.165001,-0.928982,0.141773,-0.803247,0.095792,-0.868427,0.156488,-0.990627,-0.028717,...,1.149586,1.194191,1.150906,0.673675,1.177175,1.139551,0.979461,1.160488,1.011930,0.796737
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,0.310355,-0.354439,0.272500,-0.324293,0.223226,-0.379098,0.295209,-0.329569,0.357571,-0.328131,...,-0.941361,-0.850381,-0.954068,-0.790832,-0.909188,-1.178197,-0.598379,-0.872593,-0.884629,-0.617806
2496,-1.713599,-0.713969,-1.724074,-0.729896,-1.742699,-0.933019,-1.714703,-0.734078,-1.985737,-1.007039,...,2.162044,2.196860,2.356208,2.362164,2.091965,1.912133,1.655679,2.235245,2.225369,1.987023
2497,1.894305,2.242403,1.933343,2.270921,2.015169,2.332202,1.846745,2.139874,1.850207,2.426832,...,-0.373504,-0.296734,-0.338503,-0.635766,-0.026496,-0.439205,-0.789107,-0.303868,-0.403748,0.037714
2498,-1.361596,-1.273474,-1.353029,-1.378789,-1.290359,-1.210155,-1.344453,-1.300965,-1.247424,-1.326296,...,0.929486,0.845437,0.806534,0.794281,0.920392,0.551716,1.048817,0.466375,0.540038,0.762236


### 8) Highly correlated features

In [46]:
#To create correlation matrix for X_train (features)
correlation_matrix = X_train_scaled.corr().abs()
display(correlation_matrix)

,IDV_0001_SVT3GNES12_FULLDIE_SOC_0650_MED@132110,IDV_0001_SVT3GNES12_FULLDIE_SOC_0950_MED@132110,IDV_0001_SVT3GNES12_FULLDIE_SOC_TALL_0650_MED@132110,IDV_0001_SVT3GNES12_FULLDIE_SOC_TALL_0950_MED@132110,IDV_0002_SVT3GNES12_FULLDIE_SOC_0650_MED@132110,IDV_0002_SVT3GNES12_FULLDIE_SOC_0950_MED@132110,IDV_0002_SVT3GNES12_FULLDIE_SOC_TALL_0650_MED@132110,IDV_0002_SVT3GNES12_FULLDIE_SOC_TALL_0950_MED@132110,IDV_0003_SVT3GSEM12_FULLDIE_SOC_0650_MED@132110,IDV_0003_SVT3GSEM12_FULLDIE_SOC_0950_MED@132110,...,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDCLK1,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDDATA0,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDDATA1,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TAP_DIS_STRAP_N,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TAP_ODT_EN,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_AGENT,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_PLTEN,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSCL,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSDA,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VSENSEPMAX
IDV_0001_SVT3GNES12_FULLDIE_SOC_0650_MED@132110,1.000000,0.858414,0.986821,0.849228,0.993756,0.854063,0.987386,0.849588,0.972810,0.873460,...,0.373184,0.374451,0.370373,0.377584,0.362125,0.368922,0.392636,0.383256,0.381629,0.359046
IDV_0001_SVT3GNES12_FULLDIE_SOC_0950_MED@132110,0.858414,1.000000,0.832407,0.975542,0.851902,0.990674,0.833389,0.976864,0.786876,0.979779,...,0.183277,0.184588,0.179200,0.206905,0.182987,0.192327,0.215912,0.194251,0.194457,0.181164
IDV_0001_SVT3GNES12_FULLDIE_SOC_TALL_0650_MED@132110,0.986821,0.832407,1.000000,0.852482,0.987806,0.834816,0.997611,0.849930,0.972040,0.856047,...,0.394148,0.395065,0.390792,0.397622,0.381495,0.389994,0.412221,0.405942,0.404148,0.380616
IDV_0001_SVT3GNES12_FULLDIE_SOC_TALL_0950_MED@132110,0.849228,0.975542,0.852482,1.000000,0.850099,0.977043,0.850495,0.995941,0.785327,0.964348,...,0.182832,0.184187,0.178142,0.205829,0.180734,0.193365,0.215781,0.195942,0.195946,0.181885
IDV_0002_SVT3GNES12_FULLDIE_SOC_0650_MED@132110,0.993756,0.851902,0.987806,0.850099,1.000000,0.859126,0.987459,0.848895,0.973928,0.872777,...,0.377494,0.378817,0.374250,0.381609,0.366559,0.372907,0.396731,0.387166,0.385712,0.363809
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_AGENT,0.368922,0.192327,0.389994,0.193365,0.372907,0.196504,0.382524,0.189915,0.434334,0.232863,...,0.965415,0.963168,0.964588,0.946489,0.948090,1.000000,0.941410,0.958051,0.959755,0.937245
SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_PLTEN,0.392636,0.215912,0.412221,0.215781,0.396731,0.221083,0.405493,0.212930,0.457787,0.259285,...,0.958308,0.955386,0.960003,0.946396,0.947058,0.941410,1.000000,0.958684,0.959475,0.932744
SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSCL,0.383256,0.194251,0.405942,0.195942,0.387166,0.198413,0.398783,0.192899,0.451673,0.238484,...,0.983792,0.983835,0.984263,0.959330,0.966945,0.958051,0.958684,1.000000,0.992183,0.956843
SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSDA,0.381629,0.194457,0.404148,0.195946,0.385712,0.198641,0.396970,0.192878,0.449316,0.237668,...,0.984994,0.984817,0.985141,0.958901,0.969331,0.959755,0.959475,0.992183,1.000000,0.958719


In [47]:
#To select the upper trigular matrix from the correlation matrix of features
import numpy as np
upper_tri = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape),k=1).astype(np.bool))
display(upper_tri)

<ipython-input-47-58c8dad26c5e>:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper_tri = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape),k=1).astype(np.bool))


,IDV_0001_SVT3GNES12_FULLDIE_SOC_0650_MED@132110,IDV_0001_SVT3GNES12_FULLDIE_SOC_0950_MED@132110,IDV_0001_SVT3GNES12_FULLDIE_SOC_TALL_0650_MED@132110,IDV_0001_SVT3GNES12_FULLDIE_SOC_TALL_0950_MED@132110,IDV_0002_SVT3GNES12_FULLDIE_SOC_0650_MED@132110,IDV_0002_SVT3GNES12_FULLDIE_SOC_0950_MED@132110,IDV_0002_SVT3GNES12_FULLDIE_SOC_TALL_0650_MED@132110,IDV_0002_SVT3GNES12_FULLDIE_SOC_TALL_0950_MED@132110,IDV_0003_SVT3GSEM12_FULLDIE_SOC_0650_MED@132110,IDV_0003_SVT3GSEM12_FULLDIE_SOC_0950_MED@132110,...,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDCLK1,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDDATA0,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDDATA1,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TAP_DIS_STRAP_N,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TAP_ODT_EN,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_AGENT,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_PLTEN,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSCL,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSDA,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VSENSEPMAX
IDV_0001_SVT3GNES12_FULLDIE_SOC_0650_MED@132110,NaN,0.858414,0.986821,0.849228,0.993756,0.854063,0.987386,0.849588,0.972810,0.873460,...,0.373184,0.374451,0.370373,0.377584,0.362125,0.368922,0.392636,0.383256,0.381629,0.359046
IDV_0001_SVT3GNES12_FULLDIE_SOC_0950_MED@132110,NaN,NaN,0.832407,0.975542,0.851902,0.990674,0.833389,0.976864,0.786876,0.979779,...,0.183277,0.184588,0.179200,0.206905,0.182987,0.192327,0.215912,0.194251,0.194457,0.181164
IDV_0001_SVT3GNES12_FULLDIE_SOC_TALL_0650_MED@132110,NaN,NaN,NaN,0.852482,0.987806,0.834816,0.997611,0.849930,0.972040,0.856047,...,0.394148,0.395065,0.390792,0.397622,0.381495,0.389994,0.412221,0.405942,0.404148,0.380616
IDV_0001_SVT3GNES12_FULLDIE_SOC_TALL_0950_MED@132110,NaN,NaN,NaN,NaN,0.850099,0.977043,0.850495,0.995941,0.785327,0.964348,...,0.182832,0.184187,0.178142,0.205829,0.180734,0.193365,0.215781,0.195942,0.195946,0.181885
IDV_0002_SVT3GNES12_FULLDIE_SOC_0650_MED@132110,NaN,NaN,NaN,NaN,NaN,0.859126,0.987459,0.848895,0.973928,0.872777,...,0.377494,0.378817,0.374250,0.381609,0.366559,0.372907,0.396731,0.387166,0.385712,0.363809
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_AGENT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.941410,0.958051,0.959755,0.937245
SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_PLTEN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.958684,0.959475,0.932744
SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSCL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.992183,0.956843
SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSDA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.958719


In [48]:
#To display columns with absolute correlation > 0.95
highly_corr_features_95 = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]
display(); display(highly_corr_features_95)

['IDV_0001_SVT3GNES12_FULLDIE_SOC_TALL_0650_MED@132110',
 'IDV_0001_SVT3GNES12_FULLDIE_SOC_TALL_0950_MED@132110',
 'IDV_0002_SVT3GNES12_FULLDIE_SOC_0650_MED@132110',
 'IDV_0002_SVT3GNES12_FULLDIE_SOC_0950_MED@132110',
 'IDV_0002_SVT3GNES12_FULLDIE_SOC_TALL_0650_MED@132110',
 'IDV_0002_SVT3GNES12_FULLDIE_SOC_TALL_0950_MED@132110',
 'IDV_0003_SVT3GSEM12_FULLDIE_SOC_0650_MED@132110',
 'IDV_0003_SVT3GSEM12_FULLDIE_SOC_0950_MED@132110',
 'IDV_0003_SVT3GSEM12_FULLDIE_SOC_TALL_0650_MED@132110',
 'IDV_0003_SVT3GSEM12_FULLDIE_SOC_TALL_0950_MED@132110',
 'IDV_0004_NOM3GNES12_FULLDIE_SOC_TALL_0650_MED@132110',
 'IDV_0004_NOM3GNES12_FULLDIE_SOC_TALL_0950_MED@132110',
 'IDV_0005_NOM3GNES12_FULLDIE_SOC_0650_MED@132110',
 'IDV_0005_NOM3GNES12_FULLDIE_SOC_0950_MED@132110',
 'IDV_0005_NOM3GNES12_FULLDIE_SOC_TALL_0650_MED@132110',
 'IDV_0005_NOM3GNES12_FULLDIE_SOC_TALL_0950_MED@132110',
 'IDV_0006_SVT3GISO12_FULLDIE_SOC_0650_MED@132110',
 'IDV_0006_SVT3GISO12_FULLDIE_SOC_TALL_0650_MED@132110',
 'IDV_000

In [49]:
len(highly_corr_features_95)

1013

In [50]:
#To display columns with absolute correlation > 0.90
highly_corr_features_90 = [column for column in upper_tri.columns if any(upper_tri[column] > 0.9)]
display(); display(highly_corr_features_90)

['IDV_0001_SVT3GNES12_FULLDIE_SOC_TALL_0650_MED@132110',
 'IDV_0001_SVT3GNES12_FULLDIE_SOC_TALL_0950_MED@132110',
 'IDV_0002_SVT3GNES12_FULLDIE_SOC_0650_MED@132110',
 'IDV_0002_SVT3GNES12_FULLDIE_SOC_0950_MED@132110',
 'IDV_0002_SVT3GNES12_FULLDIE_SOC_TALL_0650_MED@132110',
 'IDV_0002_SVT3GNES12_FULLDIE_SOC_TALL_0950_MED@132110',
 'IDV_0003_SVT3GSEM12_FULLDIE_SOC_0650_MED@132110',
 'IDV_0003_SVT3GSEM12_FULLDIE_SOC_0950_MED@132110',
 'IDV_0003_SVT3GSEM12_FULLDIE_SOC_TALL_0650_MED@132110',
 'IDV_0003_SVT3GSEM12_FULLDIE_SOC_TALL_0950_MED@132110',
 'IDV_0004_NOM3GNES12_FULLDIE_SOC_0950_MED@132110',
 'IDV_0004_NOM3GNES12_FULLDIE_SOC_TALL_0650_MED@132110',
 'IDV_0004_NOM3GNES12_FULLDIE_SOC_TALL_0950_MED@132110',
 'IDV_0005_NOM3GNES12_FULLDIE_SOC_0650_MED@132110',
 'IDV_0005_NOM3GNES12_FULLDIE_SOC_0950_MED@132110',
 'IDV_0005_NOM3GNES12_FULLDIE_SOC_TALL_0650_MED@132110',
 'IDV_0005_NOM3GNES12_FULLDIE_SOC_TALL_0950_MED@132110',
 'IDV_0006_SVT3GISO12_FULLDIE_SOC_0650_MED@132110',
 'IDV_0006_SVT

In [51]:
len(highly_corr_features_90)

1152

In [ ]:
#Combination of parameter with highly_corr_features_95 > 0.95/0.9

### 2) Sampling to address class imbalance

In [52]:
Counter(y_train)

Counter({1: 1269, 0: 2608})

In [54]:
from imblearn.over_sampling import SMOTE 
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from collections import Counter

#SMOTE
def SMOTE_sampling(X_train_scaled,y_train,over_amt,under_amt=None):
    """
    Can choose SMOTE or SMOTE + random undersampling
    X_train_scaled[DataFrame]: X_train after scaling
    y_train[DataFrame] : target in train data
    over_amt [int] : amount of synthetic data to be generated with SMOTE (speckle)
    under_amt[int/None]: int - amount of nonspeckle data after undersampling
                         None - No undersampling is done
    
    """
    print("Class distribution before sampling:", Counter(y_train))
    print("Ratio of class distribution before sampling :",round(Counter(y_train)[0]/Counter(y_train)[1],2))
    
    if under_amt == None: #SMOTE only
        sm = SMOTE(sampling_strategy= {1:over_amt},random_state=42)
        X_s, y_s = sm.fit_resample(X_train_scaled, y_train)
        
    else: #SMOTE + random undersampling
        sm = SMOTE(sampling_strategy={1:over_amt},random_state=42)
        under = RandomUnderSampler(sampling_strategy={0:under_amt},random_state=42)
        pipeline = Pipeline(steps=[('o', sm), ('u', under)])
        X_s, y_s = pipeline.fit_resample(X_train_scaled, y_train)    
         
    print("Class distribution after sampling:", Counter(y_s))     
    print("Ratio of class distribution after sampling :",round(Counter(y_s)[0]/Counter(y_s)[1],2)) 
    
    return X_s, y_s

#BorderlineSMOTE
def BorderlineSMOTE_sampling(X_train_scaled,y_train,over_amt,under_amt=None):
    """
    Can choose BorderlineSMOTE or BorderlineSMOTE + random undersampling
    X_train_scaled[DataFrame]: X_train after scaling
    y_train[DataFrame] : target in train data
    supporting_fs[list] : list of supporting features to be removed from X_train_scaled
    over_amt [int] : amount of synthetic data to be generated with BorderlineSMOTE
    under_amt[int/None]: int - amount of nonspeckle data after undersampling
                         None - No undersampling is done
    
    """
    print("Class distribution before sampling:", Counter(y_train))
    print("Ratio of class distribution before sampling :",round(Counter(y_train)[0]/Counter(y_train)[1],2))
    
    if under_amt == None: #BorderlineSMOTE only
        bsm = BorderlineSMOTE(sampling_strategy= {1:over_amt},random_state=42)
        X_bs, y_bs = bsm.fit_resample(X_train_scaled, y_train)
        
    else: #BorderlineSMOTE + random undersampling
        bsm = BorderlineSMOTE(sampling_strategy={1:over_amt},random_state=42)
        under = RandomUnderSampler(sampling_strategy={0:under_amt},random_state=42)
        pipeline = Pipeline(steps=[('o', bsm), ('u', under)])
        X_bs, y_bs = pipeline.fit_resample(X_train_scaled, y_train)    
         
    print("Class distribution after sampling:", Counter(y_bs))     
    print("Ratio of class distribution after sampling :",round(Counter(y_bs)[0]/Counter(y_bs)[1],2)) 
    
    return X_bs, y_bs

#ADASYN
def ADASYN_sampling(X_train_scaled,y_train,over_amt,under_amt=None):
    """
    Can choose ADASYN or ADASYN + random undersampling
    X_train_scaled[DataFrame]: X_train after scaling
    y_train[DataFrame] : target in train data
    supporting_fs[list] : list of supporting features to be removed from X_train_scaled
    over_amt [int] : amount of synthetic data to be generated with ADASYN
    under_amt[int/None]: int - amount of nonspeckle data after undersampling
                         None - No undersampling is done
    
    """
    print("Class distribution before sampling:", Counter(y_train))
    print("Ratio of class distribution before sampling :",round(Counter(y_train)[0]/Counter(y_train)[1],2))
    
    if under_amt == None: #ADASYN only
        ad = ADASYN(sampling_strategy= {1:over_amt},random_state=42)
        X_a, y_a = ad.fit_resample(X_train_scaled, y_train)
        
    else: #ADASYN + random undersampling
        ad = ADASYN(sampling_strategy={1:over_amt},random_state=42)
        under = RandomUnderSampler(sampling_strategy={0:under_amt},random_state=42)
        pipeline = Pipeline(steps=[('o', ad), ('u', under)])
        X_a, y_a = pipeline.fit_resample(X_train_scaled, y_train)    
         
    print("Class distribution after sampling:", Counter(y_a))     
    print("Ratio of class distribution after sampling :",round(Counter(y_a)[0]/Counter(y_a)[1],2)) 
    
    return X_a, y_a

In [55]:
#SMOTE
X_s, y_s = SMOTE_sampling(X_train_scaled,y_train,over_amt=2000,under_amt=2000)      

Class distribution before sampling: Counter({0: 2608, 1: 1269})
Ratio of class distribution before sampling : 2.06
Class distribution after sampling: Counter({0: 2000, 1: 2000})
Ratio of class distribution after sampling : 1.0


In [56]:
#BorderlineSMOTE
X_bs, y_bs = BorderlineSMOTE_sampling(X_train_scaled,y_train,over_amt=2000,under_amt=2000)      

Class distribution before sampling: Counter({0: 2608, 1: 1269})
Ratio of class distribution before sampling : 2.06
Class distribution after sampling: Counter({0: 2000, 1: 2000})
Ratio of class distribution after sampling : 1.0


In [57]:
#ADASYN
X_a, y_a = ADASYN_sampling(X_train_scaled,y_train,over_amt=2000,under_amt=None)      

Class distribution before sampling: Counter({0: 2608, 1: 1269})
Ratio of class distribution before sampling : 2.06
Class distribution after sampling: Counter({0: 2608, 1: 1876})
Ratio of class distribution after sampling : 1.39


In [71]:
from imblearn.over_sampling import RandomOverSampler
import warnings
warnings.filterwarnings("ignore")

#Random Under Sampling
def random_under_sampling(X_train_scaled,y_train,under_amt):
    """
    Random undersampling
    X_train_scaled[DataFrame]: X_train after scaling
    y_train[DataFrame] : target in train data
    under_amt[int]: int - amount of nonspeckle data after undersampling
    
    """
    print("Class distribution before sampling:", Counter(y_train))
    print("Ratio of class distribution before sampling :",round(Counter(y_train)[0]/Counter(y_train)[1],2))
    
    under = RandomUnderSampler(sampling_strategy={0:under_amt},random_state=42)
    X_u, y_u = under.fit_resample(X_train_scaled, y_train)
         
    print("Class distribution after sampling:", Counter(y_u))     
    print("Ratio of class distribution after sampling :",round(Counter(y_u)[0]/Counter(y_u)[1],2)) 
    
    return X_u, y_u

#Random Over Sampling
def random_over_sampling(X_train_scaled,y_train,over_amt):
    """
    Random undersampling
    X_train_scaled[DataFrame]: X_train after scaling
    y_train[DataFrame] : target in train data
    over_amt [int] : amount of synthetic data to be generated with random oversampling
    
    """
    print("Class distribution before sampling:", Counter(y_train))
    print("Ratio of class distribution before sampling :",round(Counter(y_train)[0]/Counter(y_train)[1],2))
    
    over = RandomOverSampler(sampling_strategy={0:over_amt},random_state=42)
    X_o, y_o = over.fit_resample(X_train_scaled, y_train)
         
    print("Class distribution after sampling:", Counter(y_o))     
    print("Ratio of class distribution after sampling :",round(Counter(y_o)[0]/Counter(y_o)[1],2)) 
    
    return X_o, y_o

def cou_sampling(X_train_scaled,y_train,under_amt,over_amt):
    """
    Combined random undersampling and oversampling 
    X_train_scaled[DataFrame]: X_train after scaling
    y_train[DataFrame] : target in train data
    under_amt[int]: int - amount of nonspeckle data after undersampling
    over_amt [int] : amount of synthetic data to be generated with random oversampling
    
    """
    print("Class distribution before sampling:", Counter(y_train))
    print("Ratio of class distribution before sampling :",round(Counter(y_train)[0]/Counter(y_train)[1],2))
    
    # random undersampling + oversampling
    under = RandomUnderSampler(sampling_strategy={0:under_amt},random_state=42)
    over = RandomOverSampler(sampling_strategy={1:over_amt},random_state=42)
    pipeline = Pipeline(steps=[('u', under), ('o', over)])
    X_cou, y_cou = pipeline.fit_resample(X_train_scaled, y_train)    
         
    print("Class distribution after sampling:", Counter(y_cou))     
    print("Ratio of class distribution after sampling :",round(Counter(y_cou)[0]/Counter(y_cou)[1],2)) 
    
    return X_cou, y_cou

In [72]:
X_u, y_u = random_under_sampling(X_train_scaled,y_train,under_amt=2000)

Class distribution before sampling: Counter({0: 2608, 1: 1269})
Ratio of class distribution before sampling : 2.06
Class distribution after sampling: Counter({0: 2000, 1: 1269})
Ratio of class distribution after sampling : 1.58


In [73]:
X_o, y_o = random_over_sampling(X_train_scaled,y_train,over_amt=3000)

Class distribution before sampling: Counter({0: 2608, 1: 1269})
Ratio of class distribution before sampling : 2.06
Class distribution after sampling: Counter({0: 3000, 1: 1269})
Ratio of class distribution after sampling : 2.36


In [74]:
X_cou, y_cou = cou_sampling(X_train_scaled,y_train,under_amt=2000,over_amt=3000)

Class distribution before sampling: Counter({0: 2608, 1: 1269})
Ratio of class distribution before sampling : 2.06
Class distribution after sampling: Counter({1: 3000, 0: 2000})
Ratio of class distribution after sampling : 0.67


In [ ]:
#create new column for target
# import numpy as np
# df["SPECKLE"] = np.where(df["DELTA"]==0,0,1)
# cols = df.columns.tolist()
# cols = [cols[-1]]+cols[:-1] #move speckle col to the front
# df = df.reindex(columns=cols)

In [ ]:
# df.to_csv(path+"SNR_R2_ww35.2_Speckle.csv",index=False)

In [ ]:
# supporting_fs = ['ULT@MIDAS_6261_U1', 'SORTLOT', 'SORTLOT7', 'WAFER',
#        'XLOC', 'YLOC', 'ULT@MIDAS_6261_U2', 'IB@6261[CLASSHOT]',
#        'FB@6261[CLASSHOT]','ARR_SPECKLE_SCREEN::PHYTON_X_USERFUNC_K_END_X_X_X_X_PBISTSPECKLE_INDICATOR::DUTCPUBAD_1@6261[CLASSHOT]',
#        'TEST RESULTS', 'TEST RESULTS BITS', 'TR_BITS', 'INCOMING',
#        'INCOMING BITS', 'INC_BITS', 'OUTGOING', 'OUTGOING BITS', 'OUT_BITS','DELTA']
# df1 = df.drop(supporting_fs,axis=1) 
# df1.head()

In [ ]:
import spacy
spacy.cli.download("en_core_web_sm")

### Feature Selection

### Model Building